# 3.1 线性回归

回归（regression）是能为⼀个或多个⾃变量与因变量之间关系建模的⼀类⽅法。在⾃然科学和社会科学领域，回归经常⽤来表⽰输⼊和输出之间的关系。在机器学习领域中的⼤多数任务通常都与预测（prediction）有关。当我们想预测⼀个数值时，就会涉及到回归问题。常⻅的例⼦包括：预测价格（房屋、股票等）、预测住院时间（针对住院病⼈等）、预测需求（零售销量等）。但不是所有的预测都是回归问题。在后⾯的章节中，我们将介绍分类问题。分类问题的⽬标是预
测数据属于⼀组类别中的哪⼀个。

## 线性模型

线性假设是指⽬标（房屋价格）可以表⽰为特征（⾯积和房龄）的加权和，如下⾯的式⼦

$price = w_{area}\cdot area + w_{age}\cdot age + b    (3.1.1)$

的warea和wage 称为权重（weight），权重决定了每个特征对我们预测值的影响。b称为偏置（bias）、
偏移量（offset）或截距（intercept）。偏置是指当所有特征都取值为0时，预测值应该为多少。即使现实中不
会有任何房⼦的⾯积是0或房龄正好是0年，我们仍然需要偏置项。如果没有偏置项，我们模型的表达能⼒将
受到限制。严格来说，(3.1.1)是输⼊特征的⼀个 仿射变换（affine transformation）。仿射变换的特点是通过
加权和对特征进⾏线性变换（linear transformation），并通过偏置项来进⾏平移（translation）。